Testing the celeb recognition package

https://pypi.org/project/celeb-detector/


https://www.kaggle.com/code/vinayakshanawad/celebrity-face-recognition-vggface-model/notebook


## Before running this code, install the following libraries / dependencies. Note that the face-recognition library will take several minutes to install.

pip install numpy

pip install face-recognition

pip install imutils

pip install pandas

pip install opencv-python

### For installing the Libgl package. installation might vary based on operating system:

sudo apt-get update && sudo apt-get install -y libgl1-mesa-glx



## Creating Functions

In [9]:
import os
import random
import shutil
import face_recognition
import numpy as np

# Define paths for source, training, and test directories
source_dir = "/workspace/DS4002Project3/DATA/celebrities"
train_dir = "/workspace/DS4002Project3/DATA/celebrities_train"
test_dir = "/workspace/DS4002Project3/DATA/celebrities_test"

# Number of images for training and test sets
train_count = 80
test_count = 20

# Create a function to split the data into test and training sets
def split_data():
    # Clear existing contents of training and test folders
    if os.path.exists(train_dir):
        shutil.rmtree(train_dir)
    if os.path.exists(test_dir):
        shutil.rmtree(test_dir)

    # Re-create the empty directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Iterate through each celebrity folder
    for celeb_folder in os.listdir(source_dir):
        celeb_path = os.path.join(source_dir, celeb_folder)

        if os.path.isdir(celeb_path):
            # List all image files
            images = [img for img in os.listdir(celeb_path) if img.endswith(('.jpg', '.jpeg'))]
            random.shuffle(images)  # Shuffle the images to ensure randomization

            # Ensure we have enough images
            if len(images) >= (train_count + test_count):
                # Select the first 80 images for training and the next 20 for testing
                train_images = images[:train_count]
                test_images = images[train_count:train_count + test_count]

                train_celeb_folder = os.path.join(train_dir, celeb_folder)
                test_celeb_folder = os.path.join(test_dir, celeb_folder)
                os.makedirs(train_celeb_folder, exist_ok=True)
                os.makedirs(test_celeb_folder, exist_ok=True)

                # Copy images to train and test folders without overlap
                for img in train_images:
                    shutil.copy(os.path.join(celeb_path, img), train_celeb_folder)
                for img in test_images:
                    shutil.copy(os.path.join(celeb_path, img), test_celeb_folder)
            else:
                print(f"Not enough images in {celeb_folder} for splitting.")

# Encode faces in the training set
def encode_training_faces():
    encodings = {}

    for celeb_folder in os.listdir(train_dir):
        celeb_path = os.path.join(train_dir, celeb_folder)
        if os.path.isdir(celeb_path):
            celeb_encodings = []

            # Process each image
            for img_name in os.listdir(celeb_path):
                img_path = os.path.join(celeb_path, img_name)
                img = face_recognition.load_image_file(img_path)

                # Get encodings (assuming one face per image)
                encoding = face_recognition.face_encodings(img)
                if encoding:
                    celeb_encodings.append(encoding[0])

            # Save encodings for each celebrity
            if celeb_encodings:
                encodings[celeb_folder] = np.mean(celeb_encodings, axis=0)

    return encodings

split_data()
encodings = encode_training_faces()
print("Face encodings generated for training set.")


Face encodings generated for training set.


In [10]:
# Function to count the number of images in a specified directory
def count_images_in_folder(folder_path):
    if os.path.exists(folder_path):
        return len([img for img in os.listdir(folder_path) if img.endswith(('.jpg', '.jpeg'))])
    return 0

# Count images in Jim Carrey's training and testing folders
jim_carrey_train_folder = os.path.join("/workspace/DS4002Project3/DATA/celebrities_train", "Jim-Carrey")
jim_carrey_test_folder = os.path.join("/workspace/DS4002Project3/DATA/celebrities_test", "Jim-Carrey")

num_train_images = count_images_in_folder(jim_carrey_train_folder)
num_test_images = count_images_in_folder(jim_carrey_test_folder)

print(f"Number of images in Jim Carrey's training folder: {num_train_images}")
print(f"Number of images in Jim Carrey's testing folder: {num_test_images}")


# Function to count the number of images in a specified directory
def count_images_in_folder(folder_path):
    if os.path.exists(folder_path):
        return len([img for img in os.listdir(folder_path) if img.endswith(('.jpg', '.jpeg'))])
    return 0

# Count images in Jim Carrey's training and testing folders
jim_carrey_train_folder = os.path.join("/workspace/DS4002Project3/DATA/celebrities_train", "Jackie-Chan")
jim_carrey_test_folder = os.path.join("/workspace/DS4002Project3/DATA/celebrities_test", "Jackie-Chan")

num_train_images = count_images_in_folder(jim_carrey_train_folder)
num_test_images = count_images_in_folder(jim_carrey_test_folder)

print(f"Number of images in Jackie Chan's training folder: {num_train_images}")
print(f"Number of images in Jackie Chan's testing folder: {num_test_images}")



Number of images in Jim Carrey's training folder: 80
Number of images in Jim Carrey's testing folder: 20
Number of images in Jackie Chan's training folder: 80
Number of images in Jackie Chan's testing folder: 20


In [11]:
# define function which tests for accuracy
def calculate_accuracy(encodings):
    total_images = 0
    correct_identifications = 0

    # Iterate through each celebrity folder in the test set
    for celeb_folder in os.listdir(test_dir):
        celeb_path = os.path.join(test_dir, celeb_folder)

        # Ensure it's a directory
        if os.path.isdir(celeb_path):
            # Process each image in the folder
            for test_image in os.listdir(celeb_path):
                test_image_path = os.path.join(celeb_path, test_image)

                # Load and encode test image
                test_img = face_recognition.load_image_file(test_image_path)
                test_encoding = face_recognition.face_encodings(test_img)

                if test_encoding:
                    test_encoding = test_encoding[0]

                    # Compare with known encodings
                    results = face_recognition.compare_faces(
                        list(encodings.values()), test_encoding, tolerance=0.6
                    )

                    # Find match
                    if True in results:
                        match_index = results.index(True)
                        matched_celebrity = list(encodings.keys())[match_index]

                        # Check if the identified celebrity matches the folder name
                        if matched_celebrity == celeb_folder:
                            correct_identifications += 1
                        else:
                            print(f"Incorrect: {test_image_path} identified as {matched_celebrity}")
                    else:
                        print(f"No match found for: {test_image_path}")

                # Increment total image count
                total_images += 1

    # Calculate accuracy
    accuracy = (correct_identifications / total_images) * 100 if total_images > 0 else 0
    print(f"Accuracy Rate: {accuracy:.2f}%")
    print(f"Total Images: {total_images}, Correct Identifications: {correct_identifications}")

# Run accuracy calculation
calculate_accuracy(celebrity_encodings)


Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/David-Bowie/37.jpg identified as Conan-OBrien
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Elizabeth-Olsen/100.jpg identified as Angelina-Jolie
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Elizabeth-Olsen/09.jpg identified as Angelina-Jolie
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Elizabeth-Olsen/11.jpg identified as Angelina-Jolie
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Elizabeth-Olsen/25.jpg identified as Angelina-Jolie
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Elizabeth-Olsen/28.jpg identified as Angelina-Jolie
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Elizabeth-Olsen/14.jpg identified as Angelina-Jolie
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Jim-Carrey/30.jpg identified as Jackie-Chan
Incorrect: /workspace/DS4002Project3/DATA/celebrities_test/Jim-Carrey/59.jpg identified as Danny-Pudi
Incorrect: /workspace/D